In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics, Model
from sklearn.model_selection import train_test_split
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

import cv2
from google.colab.patches import cv2_imshow
from mpl_toolkits import mplot3d
import pywt
import librosa
import librosa.display
from PIL import Image

import pylab
import glob

from __future__ import print_function
import numpy as np
import h5py
import glob
import math
import os
import shutil
from scipy import signal
from scipy.signal import butter, lfilter
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, Reshape
from keras.layers import Bidirectional, BatchNormalization, ZeroPadding1D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras import regularizers
from tensorflow.keras.layers import Layer, InputSpec
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.initializers import VarianceScaling
from keras.callbacks import CSVLogger
#from sklearn.utils.linear_assignment_ import linear_assignment
from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#import metrics
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')

from sklearn.metrics import classification_report, confusion_matrix , accuracy_score , f1_score , precision_score , recall_score

from PIL import Image

import scipy.signal as signal

In [ ]:
path_to_data = "/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders"

import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

img_dir_AS = []
for entry in os.scandir('/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders/AS(Aortic Stinosis)'):
    img_dir_AS.append(entry.path)

img_dir_MR = []
for entry in os.scandir('/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders/MR(Mitral Regurgitation)'):
    img_dir_MR.append(entry.path)

img_dir_MS = []
for entry in os.scandir('/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders/MS(Mitral Stinosis)'):
    img_dir_MS.append(entry.path)

img_dir_MVP = []
for entry in os.scandir('/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders/MVP(Mitral Valve Prolapse)'):
    img_dir_MVP.append(entry.path)

img_dir_N = []
for entry in os.scandir('/content/drive/MyDrive/Heart Disease Sound Signal Datasets Updated Folders/N'):
    img_dir_N.append(entry.path)

**STFT Preprocessing**

In [ ]:
def image_loading_and_preprocessing_stft(img_path,):
   signal , sr = librosa.load(img_path, sr=32000, mono=True)
   #melspec = librosa.feature.melspectrogram(y=signal,sr=sr)
   #melspec = librosa.power_to_db(melspec).astype(np.float32)
   stft = librosa.stft(signal)
   stft = librosa.amplitude_to_db(np.abs(stft),ref=np.max)
   Height = stft.shape[0]
   Width = stft.shape[1]
   img = Image.fromarray(stft, "I")
   img.save("my.png")
   img = np.array(img)
   arr1 = img.reshape((Height,Width,1))
   arr2 = np.array(arr1, dtype='uint8')
   resized_img = cv2.resize(arr2,(32,32))
   resized_img = resized_img.reshape((32,32,1))
   stft_rgb = cv2.cvtColor(resized_img,cv2.COLOR_GRAY2RGB)
   stft_rgb = np.array(stft_rgb)
   return stft_rgb

In [ ]:
X_stft = [ ]
y_stft = [ ]

for i in range(len(img_dir_AS)):
  image = image_loading_and_preprocessing_stft(img_dir_AS[i])
  X_stft.append(image)
  y_stft.append(0)

for i in range(len(img_dir_MR)):
  image = image_loading_and_preprocessing_stft(img_dir_MR[i])
  X_stft.append(image)
  y_stft.append(1)

for i in range(len(img_dir_MS)):
  image = image_loading_and_preprocessing_stft(img_dir_MS[i])
  X_stft.append(image)
  y_stft.append(2)

for i in range(len(img_dir_MVP)):
  image = image_loading_and_preprocessing_stft(img_dir_MVP[i])
  X_stft.append(image)
  y_stft.append(3)

for i in range(len(img_dir_N)):
  image = image_loading_and_preprocessing_stft(img_dir_N[i])
  X_stft.append(image)
  y_stft.append(4)

In [ ]:
dataset_stft = {}
dataset_stft['X'] = X_stft
dataset_stft['y'] = y_stft
dataset_stft = pd.DataFrame(dataset_stft)
#Shuffling the Dataset
df_stft = dataset_stft.sample(frac = 1)
X_shuffled_stft = df_stft['X']
y_shuffled_stft = df_stft['y']
#Removing the index and converting it into array
y_shuffled_stft = list(y_shuffled_stft)
y_shuffled_stft = np.array(y_shuffled_stft)
X_shuffled_stft = list(X_shuffled_stft)
X_shuffled_stft = np.array(X_shuffled_stft)

In [ ]:
#Saving the Feature Vector.
np.save('Heart_Disease_Prediction_STFT_Feature_Vector',X_shuffled_stft)
#Saving the Labels Vector.
np.save('Heart_Disease_Prediction_STFT_Label_Vector',y_shuffled_stft)

**Chirplet Preprocessing**

In [ ]:
from scipy.signal import chirp

In [ ]:
def image_loading_and_preprocessing_chirplet(img_path,):
   sample_s , sr = librosa.load(img_path, sr=32000, mono=True)
   melspec = librosa.feature.melspectrogram(y=sample_s,sr=sr)
   melspec = librosa.power_to_db(melspec).astype(np.float32)
   w = chirp(melspec, f0=6, f1=1, t1=10, method='linear')
   #w = w.reshape((len(w),1))
   #img_cwt = signal.cwt(sample_cwt,signal.ricker,widths)
   Height = w.shape[0]
   Width = w.shape[1]
   img = Image.fromarray(w, "I")
   img.save("my.png")
   img = np.array(img)
   arr1 = img.reshape((Height,Width,1))
   arr2 = np.array(arr1, dtype='uint8')
   resized_img = cv2.resize(arr2,(128,128))
   resized_img = resized_img.reshape((128,128,1))
   c_rgb_img = cv2.cvtColor(resized_img,cv2.COLOR_GRAY2RGB)
   c_rgb_img = np.array(c_rgb_img)
   return c_rgb_img

In [ ]:
X_c = [ ]
y_c = [ ]

for i in range(len(img_dir_AS)):
  image = image_loading_and_preprocessing_chirplet(img_dir_AS[i])
  X_c.append(image)
  y_c.append(0)

for i in range(len(img_dir_MR)):
  image = image_loading_and_preprocessing_chirplet(img_dir_MR[i])
  X_c.append(image)
  y_c.append(1)

for i in range(len(img_dir_MS)):
  image = image_loading_and_preprocessing_chirplet(img_dir_MS[i])
  X_c.append(image)
  y_c.append(2)

for i in range(len(img_dir_MVP)):
  image = image_loading_and_preprocessing_chirplet(img_dir_MVP[i])
  X_c.append(image)
  y_c.append(3)

for i in range(len(img_dir_N)):
  image = image_loading_and_preprocessing_chirplet(img_dir_N[i])
  X_c.append(image)
  y_c.append(4)

In [ ]:
dataset_c = {}
dataset_c['X'] = X_c
dataset_c['y'] = y_c
dataset_c = pd.DataFrame(dataset_c)
#Shuffling the Dataset
df_c = dataset_c.sample(frac = 1)
X_shuffled_c = df_c['X']
y_shuffled_c = df_c['y']
#Removing the index and converting it into array
y_shuffled_c = list(y_shuffled_c)
y_shuffled_c = np.array(y_shuffled_c)
X_shuffled_c = list(X_shuffled_c)
X_shuffled_c = np.array(X_shuffled_c)

In [ ]:
#Saving the Feature Vector.
np.save('Heart_Disease_Prediction_Chirplet_Feature_Vector',X_shuffled_c)
#Saving the Labels Vector.
np.save('Heart_Disease_Prediction_Chirplet_Label_Vector',y_shuffled_c)

**Loading the Preprocessed Dataset,use only this code after the first time.**

In [4]:
#Loading the Feature Vector ('x') and Labels Vector ('y).
#STFT
X_Loaded_STFT = np.load('/content/Heart_Disease_Prediction_STFT_Feature_Vector.npy')
y_Loaded_STFT = np.load('/content/Heart_Disease_Prediction_STFT_Label_Vector.npy')

#Chirplet
#Loading the Feature Vector ('x') and Labels Vector ('y).
X_Loaded_Chirplet = np.load('/content/Heart_Disease_Prediction_Chirplet_Feature_Vector.npy')
y_Loaded_Chirplet = np.load('/content/Heart_Disease_Prediction_Chirplet_Label_Vector.npy')

In [5]:
#Splitting the data into train and test
X_tr_S,X_test_S,y_tr_S,y_test_S  = train_test_split(X_Loaded_STFT , y_Loaded_STFT , test_size=0.2 , random_state=42)
X_train_S,X_val_S,y_train_S,y_val_S  = train_test_split(X_tr_S , y_tr_S , test_size=0.2 , random_state=42)

In [6]:
X_tr_C,X_test_C,y_tr_C,y_test_C  = train_test_split(X_Loaded_STFT , y_Loaded_STFT , test_size=0.2 , random_state=42)
X_train_C,X_val_C,y_train_C,y_val_C  = train_test_split(X_tr_C , y_tr_C , test_size=0.2 , random_state=42)

In [7]:
X_train_C.shape

(640, 32, 32, 3)

In [8]:
X_train_S.shape

(640, 32, 32, 3)

In [41]:
def stft_model():
    model = tf.keras.Sequential(
           [
             tf.keras.layers.InputLayer(input_shape = (32,32,3)),
             tf.keras.layers.Conv2D(60, kernel_size = (3,3),activation = 'relu',input_shape=(32,32,3)),
             tf.keras.layers.MaxPooling2D((2,2)),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Dense(128,activation = 'relu'),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(64,activation = 'relu'),
             tf.keras.layers.Dense(32,activation = 'relu'),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(5,activation = 'softmax')
           ]
    )
    return model

In [43]:
def chirplet_model():
    model = tf.keras.Sequential(
           [
             tf.keras.layers.Input((32, 32, 3)),
             tf.keras.layers.Conv2D(60, kernel_size = (3,3),activation = 'relu',input_shape=(32,32,3)),
             tf.keras.layers.MaxPooling2D((2,2)),
             tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Dense(128,activation = 'relu'),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(64,activation = 'relu'),
             tf.keras.layers.Dense(32,activation = 'relu'),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(5,activation = 'softmax')

           ]
    )
    return model

In [44]:
chirplet_model = chirplet_model()
stft_model = stft_model()

In [45]:
chirplet_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 60)        1680      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 15, 15, 60)        0         
 g2D)                                                            
                                                                 
 batch_normalization_9 (Bat  (None, 15, 15, 60)        240       
 chNormalization)                                                
                                                                 
 dense_32 (Dense)            (None, 15, 15, 128)       7808      
                                                                 
 dropout_9 (Dropout)         (None, 15, 15, 128)       0         
                                                                 
 dense_33 (Dense)            (None, 15, 15, 64)       

In [46]:
#Compiling and Training the Chirplet Model
chirplet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics='accuracy')
chirplet_model.fit(X_train_C, y_train_C, epochs=10,validation_data = (X_val_C,y_val_C))

Epoch 1/10
20/20 [==============================] - 4s 110ms/step - loss: 1.1537 - accuracy: 0.5359 - val_loss: 0.7628 - val_accuracy: 0.8125
Epoch 2/10
20/20 [==============================] - 2s 77ms/step - loss: 0.4846 - accuracy: 0.8250 - val_loss: 0.6280 - val_accuracy: 0.8562
Epoch 3/10
20/20 [==============================] - 1s 75ms/step - loss: 0.3110 - accuracy: 0.9000 - val_loss: 0.4676 - val_accuracy: 0.8625
Epoch 4/10
20/20 [==============================] - 1s 73ms/step - loss: 0.2217 - accuracy: 0.9125 - val_loss: 0.4831 - val_accuracy: 0.8875
Epoch 5/10
20/20 [==============================] - 2s 76ms/step - loss: 0.1495 - accuracy: 0.9563 - val_loss: 0.3918 - val_accuracy: 0.8938
Epoch 6/10
20/20 [==============================] - 2s 82ms/step - loss: 0.0937 - accuracy: 0.9750 - val_loss: 0.3955 - val_accuracy: 0.8813
Epoch 7/10
20/20 [==============================] - 2s 120ms/step - loss: 0.0571 - accuracy: 0.9875 - val_loss: 0.4730 - val_accuracy: 0.8813
Epoch 8/10


In [48]:
#Compiling and Training the STFT Model
stft_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics='accuracy')
stft_model.fit(X_train_S, y_train_S, epochs=10,validation_data = (X_val_S,y_val_S))

Epoch 1/10
20/20 [==============================] - 8s 91ms/step - loss: 1.0109 - accuracy: 0.5953 - val_loss: 1.0439 - val_accuracy: 0.6625
Epoch 2/10
20/20 [==============================] - 2s 111ms/step - loss: 0.4533 - accuracy: 0.8219 - val_loss: 0.5470 - val_accuracy: 0.8500
Epoch 3/10
20/20 [==============================] - 2s 83ms/step - loss: 0.3174 - accuracy: 0.8922 - val_loss: 0.4895 - val_accuracy: 0.8625
Epoch 4/10
20/20 [==============================] - 2s 76ms/step - loss: 0.2039 - accuracy: 0.9297 - val_loss: 0.4517 - val_accuracy: 0.8750
Epoch 5/10
20/20 [==============================] - 2s 108ms/step - loss: 0.1315 - accuracy: 0.9578 - val_loss: 0.3757 - val_accuracy: 0.9125
Epoch 6/10
20/20 [==============================] - 2s 113ms/step - loss: 0.0963 - accuracy: 0.9734 - val_loss: 0.3905 - val_accuracy: 0.8875
Epoch 7/10
20/20 [==============================] - 2s 98ms/step - loss: 0.0885 - accuracy: 0.9719 - val_loss: 0.4229 - val_accuracy: 0.9000
Epoch 8/10

In [14]:
 # Extract features or predictions from the final layers of both models
chirplet_features = chirplet_model.predict(X_train_C)
stft_features = stft_model.predict(X_train_S)

# Concatenate or combine the features
combined_features = np.concatenate((X_train_C, X_train_S), axis=1)
combined_labels = np.concatenate((y_train_C , y_train_S), axis=0)

combined_validation_features = np.concatenate((X_val_C, X_val_S), axis=1)
combined_validation_labels = np.concatenate((y_val_C , y_val_S), axis=0)

# Define and train a fully connected network on the combined features
def create_fully_connected_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(combined_features.shape[1],)),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(32,activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # Adjust num_classes based on your task
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

fully_connected_model = create_fully_connected_model()

# Train the fully connected model
fully_connected_model.fit(combined_features, combined_labels, epochs=10)  # Replace labels

# Evaluate the ensemble model on validation or test data
evaluation_result = fully_connected_model.evaluate(combined_validation_features, combined_validation_features)  # Replace validation_data and validation_labels


RuntimeError: ignored

In [ ]:
# Make predictions using the ensemble model on new data
predictions = fully_connected_model.predict(new_data)